# <strong><u> 3a: Event Finding Notebook
This notebook is designed with the purpose of identifying gesture events based on kinematic features of the motion tracking data.

In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks

### <strong> Import Data

In [ ]:
PARTICIPANT = 'TEST_VIDEO'
df = pd.read_csv(f"../Upsampled/{PARTICIPANT}_processed_data.csv")
df.head()

### <Strong> Gather Peaks in the Speed Curve

In [ ]:
# Using scipy's find_peaks method to find peaks in the 'right_wrist_speed' column and save 
peaks, _ = find_peaks(df['right_wrist_speed'], height=(np.percentile(df['right_wrist_speed'], 70)))  # height parameter can be adjusted based on your specific needs
# Keep the time values of the peaks
peak_times = df['time_ms'][peaks]
peak_times

### <strong> Find the onset/offset of Peaks 

In [ ]:
# Initialize empty lists to store onset and offset points
onsets = []
offsets = []

# Define a threshold for onset and offset detection; this can be adjusted based on your needs
onset_threshold = np.percentile(df['right_wrist_speed'], 5)
offset_threshold = np.percentile(df['right_wrist_speed'], 5)

# Loop through each peak to find the corresponding onset and offset
for peak in peaks:
    # Find onset by iterating backwards from the peak until the speed drops below the threshold
    onset = peak
    while onset > 0 and df['right_wrist_speed'][onset] > onset_threshold:
        onset -= 1
    onsets.append(df['time_ms'][onset])  # Store the actual time value instead of the index
    
    # Find offset by iterating forwards from the peak until the speed drops below the threshold
    offset = peak
    while offset < len(df) - 1 and df['right_wrist_speed'][offset] > offset_threshold:
        offset += 1
    offsets.append(df['time_ms'][offset])  # Store the actual time value instead of the index

### <strong> Event Composition
Matching pairs of onset and offsets and combining to form full gesture events.

In [ ]:
# Pair each onset with the next offset to form complete gestures
events = list(zip(onsets, offsets))
events_df = pd.DataFrame(events, columns=['onset', 'offset'])

# Keep only the first instance of each onset offset pair
events_df.drop_duplicates(subset=['onset', 'offset'], inplace=True)

# Calculate the duration of each gesture
events_df['duration'] = events_df['offset'] - events_df['onset']

# Reset the DataFrame's index
events_df.reset_index(drop=True, inplace=True)

# Asign gesture id to each gesture
events_df['gesture_id'] = events_df.index + 1

# If the offset of one gesture is within 50 ms of the onset of the next, combine them into one gesture
for i in range(len(events_df) - 1):
    if events_df['onset'][i + 1] - events_df['offset'][i] < 50:
        events_df['offset'][i] = events_df['offset'][i + 1]
        events_df['duration'][i] = events_df['offset'][i] - events_df['onset'][i]
        events_df['gesture_id'][i + 1] = events_df['gesture_id'][i]

# Using the events_df DataFrame, create a new column in the original dataframe and apply the gesture_id to where they belong in the original dataframe
df['gesture_id'] = np.nan

for i in range(len(events_df)):
    start = events_df['onset'][i]
    end = events_df['offset'][i]
    df['gesture_id'] = np.where((df['time_ms'] >= start) & (df['time_ms'] <= end), events_df['gesture_id'][i], df['gesture_id'])

In [ ]:
# Plot the gesture_id column to see the results
import plotly.graph_objects as go

fig = go.Figure()

# Add the first trace with the gesture_id column
fig.add_trace(go.Scatter(x=df['time_ms'], y=df['gesture_id'], name='Gesture ID'))

# Add the second trace with the right_wrist_speed column
fig.add_trace(go.Scatter(x=df['time_ms'], y=df['right_wrist_speed'], name='Right Wrist Speed', yaxis='y2'))

# Set the layout with two y-axes
fig.update_layout(
       yaxis=dict(
              title='Gesture ID',
              titlefont=dict(color='blue'),
              tickfont=dict(color='blue')
       ),
       yaxis2=dict(
              title='Right Wrist Speed',
              titlefont=dict(color='red'),
              tickfont=dict(color='red'),
              overlaying='y',
              side='right'
       ),
       # change size of the figure
       width=1000,
       height=800
)

fig.show()

In [ ]:
# Save the new annotated dataframe as a csv file
df.to_csv(f'../Events/{PARTICIPANT}_events_found.csv', index=False)